In [1]:
1+1

2

hola mundo

In [6]:
import sqlite3

# Connect to SQLite database (or create it)
conn = sqlite3.connect('ink_material_compatibility.db')
cursor = conn.cursor()

# Execute SQL code to create tables
cursor.execute('''CREATE TABLE Inks (
    InkID INT PRIMARY KEY,
    Viscosity FLOAT,
    DryingTime FLOAT,
    PigmentType VARCHAR(50),
    Color VARCHAR(20),
    ChemicalComposition VARCHAR(255),
    PHLevel FLOAT
)''')

cursor.execute('''CREATE TABLE Materials (
    MaterialID INT PRIMARY KEY,
    SurfaceEnergy FLOAT,
    Porosity FLOAT,
    Texture VARCHAR(50),
    MaterialType VARCHAR(50),
    Thickness FLOAT,
    TemperatureResistance FLOAT
)''')

cursor.execute('''CREATE TABLE Compatibility (
    CompatibilityID INT PRIMARY KEY,
    InkID INT,
    MaterialID INT,
    CompatibilityScore FLOAT,
    FOREIGN KEY (InkID) REFERENCES Inks(InkID),
    FOREIGN KEY (MaterialID) REFERENCES Materials(MaterialID)
)''')

# Commit changes and close connection
conn.commit()
conn.close()


OperationalError: table Inks already exists

In [1]:
import pandas as pd
import numpy as np

# Number of samples
num_inks = 100
num_materials = 100
num_compatibility = 1000

# List of material names
material_names = [
    'Polyethylene', 'Polypropylene', 'Polyvinyl Chloride', 'Polyethylene Terephthalate', 
    'Polystyrene', 'Nylon', 'Acrylic', 'Polycarbonate', 'Silicone', 'Polyurethane', 
    'Glass', 'Stainless Steel', 'Aluminum', 'Copper', 'Brass', 
    'Paper', 'Cardboard', 'Textile', 'Wood', 'Leather'
]

# If the number of materials is greater than the list of names, repeat the names
if num_materials > len(material_names):
    material_names = np.random.choice(material_names, num_materials, replace=True)
else:
    material_names = np.random.choice(material_names, num_materials, replace=False)

# Generate Inks data
inks = pd.DataFrame({
    'InkID': range(1, num_inks + 1),
    'Viscosity': np.random.uniform(1, 10, num_inks),
    'DryingTime': np.random.uniform(0.5, 5, num_inks),
    'PigmentType': np.random.choice(['Organic', 'Inorganic'], num_inks),
    'Color': np.random.choice(['Red', 'Blue', 'Green', 'Black', 'Yellow'], num_inks),
    'ChemicalComposition': np.random.choice(['C1', 'C2', 'C3', 'C4'], num_inks),
    'PHLevel': np.random.uniform(3, 10, num_inks)
})

# Generate Materials data
materials = pd.DataFrame({
    'MaterialID': range(1, num_materials + 1),
    'MaterialName': material_names,
    'SurfaceEnergy': np.random.uniform(20, 60, num_materials),
    'Porosity': np.random.uniform(0, 1, num_materials),
    'Texture': np.random.choice(['Smooth', 'Rough', 'Matte', 'Glossy'], num_materials),
    'MaterialType': np.random.choice(['Plastic', 'Metal', 'Paper', 'Glass', 'Fabric'], num_materials),
    'Thickness': np.random.uniform(0.1, 5, num_materials),
    'TemperatureResistance': np.random.uniform(-20, 120, num_materials)
})

# Generate Compatibility data
compatibility = pd.DataFrame({
    'CompatibilityID': range(1, num_compatibility + 1),
    'InkID': np.random.choice(inks['InkID'], num_compatibility),
    'MaterialID': np.random.choice(materials['MaterialID'], num_compatibility),
    'CompatibilityScore': np.random.uniform(0, 100, num_compatibility)
})

# Save to CSV files (optional)
inks.to_csv('inks.csv', index=False)
materials.to_csv('materials.csv', index=False)
compatibility.to_csv('compatibility.csv', index=False)


In [2]:
# Load data from CSV files (if saved)
inks = pd.read_csv('inks.csv')
materials = pd.read_csv('materials.csv')
compatibility = pd.read_csv('compatibility.csv')

# Example of merging tables to get full compatibility information
full_data = compatibility.merge(inks, on='InkID').merge(materials, on='MaterialID')

# Display the first few rows of the merged data
print(full_data.head())


   CompatibilityID  InkID  MaterialID  CompatibilityScore  Viscosity  \
0                1     54           5           22.383632   9.459846   
1                2     22          30           83.749007   6.119261   
2                3     75          38           90.420133   8.394775   
3                4     14          79            3.388854   3.011364   
4                5     90          50           10.041665   5.047390   

   DryingTime PigmentType  Color ChemicalComposition   PHLevel  \
0    3.979665   Inorganic    Red                  C2  8.786186   
1    0.685389     Organic   Blue                  C3  5.634324   
2    2.892286     Organic  Black                  C2  3.115803   
3    4.288763     Organic   Blue                  C3  9.884656   
4    4.683262     Organic   Blue                  C3  9.071360   

      MaterialName  SurfaceEnergy  Porosity Texture MaterialType  Thickness  \
0            Brass      40.571039  0.523192   Matte        Metal   2.677042   
1  Stainless

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Preparing data for the model
X = full_data[['Viscosity', 'DryingTime', 'PHLevel', 'SurfaceEnergy', 'Porosity', 'Thickness', 'TemperatureResistance']]
y = full_data['CompatibilityScore']

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the model
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Making predictions
y_pred = model.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 1643.2815302165377


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Example URL (replace with actual URL of a material list page)
url = "https://en.wikipedia.org/wiki/Thermoplastic#"

# Send a request to fetch the content of the page
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all material names (this example assumes materials are in list items)
materials_list = soup.find_all('li')

# Extract and clean the text
materials = [item.get_text() for item in materials_list]

# Convert to DataFrame
materials_df = pd.DataFrame(materials, columns=['MaterialName'])

# Save to CSV
materials_df.to_csv('scraped_materials.csv', index=False)


In [7]:
import requests
import pandas as pd

# Define your API key
api_key = 'your_api_key_here'

# API endpoint for Materials Project
url = f'https://materialsproject.org/rest/v2/materials/mp-1234/vasp'

# Headers
headers = {
    'X-API-KEY': api_key
}

# Make the request
response = requests.get(url, headers=headers)

# Parse JSON response
data = response.json()

# Extract data into a DataFrame
materials_df = pd.DataFrame(data['response'])
materials_df.to_csv('materials_project_data.csv', index=False)

# Display data
print(materials_df.head())


KeyError: 'response'

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to scrape material data from a specific URL on MatWeb
def scrape_matweb_materials(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    materials = []
    
    # Find the table containing the material data
    table = soup.find('table', {'class': 'tablesorter'})
    if table:
        headers = [th.get_text().strip() for th in table.find_all('th')]
        rows = table.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.get_text().strip() for ele in cols]
            if cols:
                materials.append(cols)
    
    return headers, materials

# Example URL (replace with the actual URL for the plastics section)
url = "https://www.matweb.com/search/MaterialGroupSearch.aspx"

headers, materials = scrape_matweb_materials(url)

# Convert to DataFrame
if headers and materials:
    df = pd.DataFrame(materials, columns=headers)
    df.to_csv('matweb_plastics.csv', index=False)
    print(df.head())
else:
    print("No data found.")


UnboundLocalError: cannot access local variable 'headers' where it is not associated with a value

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Example URL for chemical compatibility (replace with actual URL)
url = "https://www.coleparmer.com/Chemical-Resistance"

# Function to scrape compatibility data
def scrape_compatibility_data(url, chemical, material):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Example of locating data, modify according to actual HTML structure
    compatibility_table = soup.find('table', {'class': 'compatibility-table'})
    rows = compatibility_table.find_all('tr')
    
    data = []
    for row in rows[1:]:  # Skip header row
        cols = row.find_all('td')
        if cols:
            data.append({
                'Chemical': cols[0].get_text().strip(),
                'Material': cols[1].get_text().strip(),
                'Compatibility': cols[2].get_text().strip()
            })
    
    return pd.DataFrame(data)

# Scrape data
chemical = "Acetone"
material = "HDPE"
compatibility_data = scrape_compatibility_data(url, chemical, material)

# Save to CSV
compatibility_data.to_csv('chemical_compatibility.csv', index=False)

# Display data
print(compatibility_data.head())


AttributeError: 'NoneType' object has no attribute 'find_all'

In [1]:
     InkID  Pigment  Solvent  Resin  Additive
0   Ink 1  0.2345  0.4121  0.2000  0.1534
1   Ink 2  0.4567  0.2345  0.1500  0.1598
2   Ink 3  0.1234  0.5678  0.1800  0.1288
3   Ink 4  0.3456  0.1890  0.2200  0.2454
4   Ink 5  0.6789  0.1234  0.1000  0.0977
..     ...     ...     ...     ...     ...
19  Ink 20  0.5432  0.3211  0.0800  0.0557

SyntaxError: invalid syntax (1784529025.py, line 1)

In [2]:
pip install faker

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 1.3 MB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.8 MB 5.0 MB/s eta 0:00:01
   ----------------- ---------------------- 0.8/1.8 MB 7.0 MB/s eta 0:00:01
   ----------------------------- ---------- 1.3/1.8 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 8.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from faker import Faker

fake = Faker()

# Define the list of plastics
plastics = ['Polypropylene', 'HDPE', 'PVC', 'PET', 'LDPE', 'PPSU', 'ABS', 'Nylon', 'Teflon', 'Polycarbonate', 'PMMA', 'MABS', 'SAN', 'PEI', 'PE', 'POM', 'PPO', 'PPO+20GF']

# Define the list of chemicals
chemicals = ['Acetone', 'Ethanol', 'Toluene', 'Xylene', 'MEK', 'IPA', 'Glycerin', 'Diethylene Glycol', 'Triethylene Glycol', 'Water', 'UV Stabilizer', 'Silicone Oil', 'Phthalate', 'Surfactant', 'Antioxidant']

# Create a dictionary to store the compatibility data
compatibility_data = {}

# Loop through each plastic and generate a random compatibility score for each chemical
for plastic in plastics:
    compatibility_data[plastic] = [fake.random_int(min=0, max=100, step=1) / 100 for _ in range(len(chemicals))]

# Create a Pandas DataFrame from the compatibility data
df = pd.DataFrame(compatibility_data, index=chemicals).T

# Print the resulting table
print(df)

               Acetone  Ethanol  Toluene  Xylene   MEK   IPA  Glycerin  \
Polypropylene     0.79     0.41     0.44    0.20  0.56  0.30      0.37   
HDPE              0.44     0.38     0.26    0.54  0.37  0.53      0.43   
PVC               0.57     0.86     0.28    0.57  0.83  0.58      0.82   
PET               0.03     0.98     0.61    0.80  0.24  0.46      0.60   
LDPE              0.11     0.78     0.22    0.27  0.79  0.22      0.04   
PPSU              0.91     0.36     0.98    0.34  0.12  0.34      0.12   
ABS               0.99     0.20     0.11    0.94  0.87  0.91      0.16   
Nylon             0.20     0.72     0.36    0.46  0.56  0.44      0.69   
Teflon            0.56     1.00     0.84    0.61  0.09  0.31      0.20   
Polycarbonate     0.95     0.18     0.54    0.54  0.07  0.77      0.52   
PMMA              0.04     0.38     0.65    0.42  0.59  0.38      0.76   
MABS              0.75     0.48     0.13    0.74  0.84  1.00      0.85   
SAN               0.69     0.08     0.

In [4]:
import pandas as pd
import requests

# Define the list of plastics
plastics = ['Polypropylene', 'HDPE', 'PVC', 'PET', 'LDPE', 'PPSU', 'ABS', 'Nylon', 'Teflon', 'Polycarbonate', 'PMMA', 'MABS', 'SAN', 'PEI', 'PE', 'POM', 'PPO', 'PPO+20GF']

# Define the list of chemicals
chemicals = ['Acetone', 'Ethanol', 'Toluene', 'Xylene', 'MEK', 'IPA', 'Glycerin', 'Diethylene Glycol', 'Triethylene Glycol', 'Water', 'UV Stabilizer', 'Silicone Oil', 'Phthalate', 'Surfactant', 'Antioxidant']

# Create a dictionary to store the compatibility data
compatibility_data = {}

# Loop through each plastic and retrieve compatibility data from the CompTox Chemistry Dashboard
for plastic in plastics:
    compatibility_data[plastic] = []
    for chemical in chemicals:
        # Make a request to the CompTox Chemistry Dashboard API
        url = f"https://comptox.epa.gov/dashboard/api/chemical/{chemical}/exposure/product-use-categories"
        response = requests.get(url)
        data = response.json()
        
        # Extract the compatibility score from the response data
        compatibility_score = 0
        for category in data['categories']:
            if category['category'] == plastic:
                compatibility_score = category['score']
                break
        
        # Add the compatibility score to the dictionary
        compatibility_data[plastic].append(compatibility_score)

# Create a Pandas DataFrame from the compatibility data
df = pd.DataFrame(compatibility_data, index=chemicals).T

# Print the resulting table
print(df)

SSLError: HTTPSConnectionPool(host='comptox.epa.gov', port=443): Max retries exceeded with url: /dashboard/api/chemical/Acetone/exposure/product-use-categories (Caused by SSLError(SSLError(1, '[SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED] unsafe legacy renegotiation disabled (_ssl.c:1000)')))